In [1]:
# centroids generated from this step do not go through the pixel filter!
# but centroids from first half were generated via this step soooooooooooooooooo
# I could generate some centroids from the first half and compare to current values
# looks like I already generated those values
#so why is second half getting flagged? differently
#doesnt explain why there are half differences but does explain the fact the flower is getting picked up p frequ. 
#one filter only saves centroid values for objects with a contour larger than x
#the other filter saves in and out values for 

In [2]:
import time
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import os, errno
import glob

In [3]:
def FrameSequence(file_name, path):
    video = [(os.path.join(root, file), file[0:-4]) 
        for root, dirs, files in os.walk(path) 
        for file in files 
        if file.endswith(file_name + '.mp4')]
    cam = cv2.VideoCapture(video[0][0])
    cam.set(1,0)
    frame_sequence = list(range(0,int(cam.get(cv2.CAP_PROP_FRAME_COUNT))))

    return frame_sequence

In [4]:
file_truet = r"C:\Users\Daniellab\Desktop\Jorge_Analysis\In_Out_Refined\Post\AllMoths.csv"
location = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Centroids_Raw'
outpath = r'C:\Users\Daniellab\Desktop\Jorge_Analysis\Centroids_Final\Raw_NG'
video = r"C:\Users\Daniellab\Desktop\Jorge_Analysis\Videos"

In [5]:
# open data frame with all true trial info
# pull out each video based on a name from a list
# for each name pull out all start and end trials that also have true trials
# choose the start and end frame for the first and last entry
# use those integers to loop through (one loop for first centroid and one for last centroid - seperate cells to generate seperate csv w/o altering the visit type)
#     end_frame: list_start/finish[len(list_start/finish)-1]

In [6]:
dFrame = pd.read_csv(file_truet)
list_videos = dFrame.name.unique()

for name in list_videos:
    i = 0
    In_frame = list(dFrame[(dFrame.name == name) & (dFrame.In_Frame.notnull())].In_Frame)
    
    Out_frame = list(dFrame[(dFrame.name == name) & (dFrame.Out_Frame.notnull())].Out_Frame)
    print(len(In_frame), name)

3 Experimental_1
2 Experimental_2
1 Experimental_3
2 Experimental_4
2 Experimental_5
4 Experimental_7
1 Pseudo_1
5 Pseudo_2
1 Sham_10
2 Sham_11
2 Sham_14
5 Sham_15
2 Sham_16
1 Sham_17
1 Sham_18
7 Sham_1
3 Sham_2
4 Sham_3
3 Sham_4
6 Sham_6
7 Sham_7


In [7]:
dFrame = pd.read_csv(file_truet)
list_videos = dFrame.name.unique()
list_videos

array(['Experimental_1', 'Experimental_2', 'Experimental_3',
       'Experimental_4', 'Experimental_5', 'Experimental_7', 'Pseudo_1',
       'Pseudo_2', 'Sham_10', 'Sham_11', 'Sham_14', 'Sham_15', 'Sham_16',
       'Sham_17', 'Sham_18', 'Sham_1', 'Sham_2', 'Sham_3', 'Sham_4',
       'Sham_6', 'Sham_7'], dtype=object)

In [8]:
In_frame

[54387, 56021, 59103, 59901, 63837, 64471, 64994]

In [9]:
visit_number = 0
visit_number_list = []
visit_number_list.append(visit_number)
visit_number_list

[0]

In [17]:
i = 0
for name in list_videos:

    In_frame = list(dFrame[(dFrame.name == name) & (dFrame.In_Frame.notnull())].In_Frame)
    
    Out_frame = list(dFrame[(dFrame.name == name) & (dFrame.Out_Frame.notnull())].Out_Frame)
    print(len(In_frame), name)
    
    print(name)
    frames = FrameSequence(name, video)
    i = 0
    df  = pd.read_csv(location + "\\" + name + ".csv")
    for visit in Out_frame:
        visit_number = i
        print(visit_number)
        start_frame = In_frame[i]
        end_frame = visit
        frameNum = frames[start_frame:end_frame]
        contours = []
        centroids_x = []
        centroids_y = []
        contours_hull = []
        centroids_hull_x = []
        centroids_hull_y = []
        frame_ = []

        
        name_ = str(name) + "_" + str(start_frame)
        i+=1
        for frame in frameNum:
            cent_x = df[(df.index == frame)].Centroid_hull_x
            cent_y = df[(df.index == frame)].Centroid_hull_y
            centroids_hull_x.append(np.squeeze(np.asarray(cent_x)))
            centroids_hull_y.append(np.squeeze(np.asarray(cent_y)))
            frame_.append(frame)

        gauss = signal.gaussian(10,3)
        x_gauss = signal.convolve(centroids_hull_x, gauss, mode='same', method = 'direct') / gauss.sum()
        y_gauss = signal.convolve(centroids_hull_y, gauss, mode = 'same', method = 'direct') / gauss.sum()

        new_df = (pd.DataFrame({'FrameNumber' : frame_, 'Name': name_, 'X_position' : x_gauss, 
                                    'Y_position' : y_gauss, 'visit_number': visit_number}))

        new_df.to_csv(outpath + "//" + 'Centroids_Final_' + name_ + '.csv')

3 Experimental_1
Experimental_1
0
1
2
2 Experimental_2
Experimental_2
0
1
1 Experimental_3
Experimental_3
0
2 Experimental_4
Experimental_4
0
1
2 Experimental_5
Experimental_5
0
1
4 Experimental_7
Experimental_7
0
1
2
3
1 Pseudo_1
Pseudo_1
0
5 Pseudo_2
Pseudo_2
0
1
2
3
4
1 Sham_10
Sham_10
0
2 Sham_11
Sham_11
0
1
2 Sham_14
Sham_14
0
1
5 Sham_15
Sham_15
0
1
2
3
4
2 Sham_16
Sham_16
0
1
1 Sham_17
Sham_17
0
1 Sham_18
Sham_18
0
7 Sham_1
Sham_1
0
1
2
3
4
5
6
3 Sham_2
Sham_2
0
1
2
4 Sham_3
Sham_3
0
1
2
3
3 Sham_4
Sham_4
0
1
2
6 Sham_6
Sham_6
0
1
2
3
4
5
7 Sham_7
Sham_7
0
1
2
3
4
5
6


In [ ]:
asdf

In [ ]:
gauss = signal.gaussian(3,3)
x_gauss = signal.convolve(centroids_hull_x, gauss, mode='same', method = 'direct') / gauss.sum()
y_gauss = signal.convolve(centroids_hull_y, gauss, mode = 'same', method = 'direct') / gauss.sum()

In [ ]:
for name in list_videos:

    In_frame = list(dFrame[(dFrame.name == name) & (dFrame.In_Frame.notnull())].In_Frame)
    
    Out_frame = list(dFrame[(dFrame.name == name) & (dFrame.Out_Frame.notnull())].Out_Frame)
    print(len(In_frame), name)
    
    print(name)
    frames = FrameSequence(name, video)

    start_frame = In_frame[0]-5
    end_frame = Out_frame[0]+5
    frameNum = frames[start_frame:end_frame]
    contours = []
    centroids_x = []
    centroids_y = []
    contours_hull = []
    centroids_hull_x = []
    centroids_hull_y = []
    frame_ = []

    df  = pd.read_csv(location + "\\" + name + ".csv")

    
    for frame in frameNum:
        cent_x = df[(df.index == frame)].Centroid_hull_x
        cent_y = df[(df.index == frame)].Centroid_hull_y
        centroids_hull_x.append(np.squeeze(np.asarray(cent_x)))
        centroids_hull_y.append(np.squeeze(np.asarray(cent_y)))
        frame_.append(frame)

    new_df = (pd.DataFrame({'FrameNumber' : frame_, 'Name': name, 'X_position' : centroids_hull_x, 
                                'Y_position' : centroids_hull_y}))

    new_df.to_csv(outpath + "//" + 'Centroids_Final_' + name + '.csv')

In [ ]:
cent_x

In [ ]:
second_last = ['L0.1_c-3_m10', 'L0.1_c-3_m12', 'L0.1_c-3_m20', 'L0.1_c-3_m21',
       'L0.1_c-3_m22', 'L0.1_c-3_m23', 'L0.1_c-3_m24', 'L0.1_c-3_m25',
       'L0.1_c-3_m27', 'L0.1_c-3_m2', 'L0.1_c-3_m32', 'L0.1_c-3_m34',
       'L0.1_c-3_m37', 'L0.1_c-3_m38', 'L0.1_c-3_m39', 'L0.1_c-3_m40',
       'L0.1_c-3_m41', 'L0.1_c-3_m43', 'L0.1_c-3_m44', 'L0.1_c-3_m5',
       'L0.1_c-3_m8', 'L50_c-3_m10', 'L50_c-3_m12', 'L50_c-3_m13',
       'L50_c-3_m14', 'L50_c-3_m15', 'L50_c-3_m21', 'L50_c-3_m22',
       'L50_c-3_m24', 'L50_c-3_m25', 'L50_c-3_m26', 'L50_c-3_m2',
       'L50_c-3_m30', 'L50_c-3_m32', 'L50_c-3_m33', 'L50_c-3_m34',
       'L50_c-3_m35', 'L50_c-3_m37', 'L50_c-3_m38', 'L50_c-3_m39',
       'L50_c-3_m45', 'L50_c-3_m6', 'L50_c-3_m9']

In [ ]:
second_last = [('L0.1_c-3_m10', -2), ('L0.1_c-3_m12', -2), ('L0.1_c-3_m20', -1), ('L0.1_c-3_m21', -1),
       ('L0.1_c-3_m22', -1), ('L0.1_c-3_m23', -2), ('L0.1_c-3_m25', -1), (
       'L0.1_c-3_m27', -5), ('L0.1_c-3_m2', -1), ('L0.1_c-3_m32', -1), ('L0.1_c-3_m34', -2),
       ('L0.1_c-3_m37', 2), ('L0.1_c-3_m38', -1), ('L0.1_c-3_m40', -2),
       ('L0.1_c-3_m41', -2), ('L0.1_c-3_m43', -1), ('L0.1_c-3_m44', -1), ('L0.1_c-3_m45', -1),
       ('L0.1_c-3_m46', -1), ('L0.1_c-3_m47', -1), ('L0.1_c-3_m48', -1), ('L0.1_c-3_m49', -1),
       ('L0.1_c-3_m50', -1), ('L0.1_c-3_m54', -1), ('L0.1_c-3_m57', -1), ('L0.1_c-3_m5', -1),
       ('L0.1_c-3_m8', 11), ('L50_c-3_m10', -1), ('L50_c-3_m12', -3), ('L50_c-3_m13', -1),
       ('L50_c-3_m14', -1), ('L50_c-3_m15', -1), ('L50_c-3_m21', -1), ('L50_c-3_m22', -1),
       ('L50_c-3_m26', -1), ('L50_c-3_m2', -1),
       ('L50_c-3_m30', -2), ('L50_c-3_m32', -1), ('L50_c-3_m33', -1), ('L50_c-3_m34', -1),
       ('L50_c-3_m35', -4), ('L50_c-3_m37', -2), ('L50_c-3_m38', 1),  ('L50_c-3_m39', -1),
       ('L50_c-3_m45', -9), ('L50_c-3_m6', -1), ('L50_c-3_m9', -1), ('L50_c-3_m49', -1), ('L50_c-3_m50', -1), ('L50_c-3_m51', -1),
       ('L50_c-3_m58',-1)]

In [ ]:
second_last_names = []
for element in second_last:
    second_last_names.append(element[0])
second_last_names

In [ ]:
for name in second_last:
    index = name[1]
    name = name[0]
    
    In_frame = list(dFrame[(dFrame.name == name) & (dFrame.In_Frame.notnull())].In_Frame)
    
    Out_frame = list(dFrame[(dFrame.name == name) & (dFrame.Out_Frame.notnull())].Out_Frame)


    print(name)
    frames = FrameSequence(name, video)

    start_frame = In_frame[index]+20
    end_frame = Out_frame[index]-20
    frameNum = frames[start_frame:end_frame]
    contours = []
    centroids_x = []
    centroids_y = []
    contours_hull = []
    centroids_hull_x = []
    centroids_hull_y = []
    frame_ = []

    
    df  = pd.read_csv(r"C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Step1_Tanvi_Method" + "\\" + name + ".csv")

    
    for frame in frameNum:
        cent_x = list(df[(df.index == frame)].Centroid_hull_x)
        cent_y = list(df[(df.index == frame)].Centroid_hull_y)
        centroids_hull_x.append(np.squeeze(np.asarray(cent_x)))
        centroids_hull_y.append(np.squeeze(np.asarray(cent_y)))
        frame_.append(frame)
    new_df = (pd.DataFrame({'FrameNumber' : frame_, 'Name': name, 'X_position' : centroids_hull_x, 
                                'Y_position' : centroids_hull_y}))
    second_last_outpath = r"C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Motion_Analysis\First_Last_Centroids\Second_Last"
    new_df.to_csv(outpath + "//" + 'Final_position_centroid_method_Last_SecondTrial_' + name + '.csv')